This notebook demonstrates how to use the pre-trained models to predict emission intensity on an arbitrary UCNP. 

In [12]:
from NanoParticleTools.machine_learning.data import SummedWavelengthRangeLabelProcessor
from NanoParticleTools.machine_learning.models.hetero.intra_inter_model import HeteroDCVModel
from NanoParticleTools.machine_learning.models.hetero.intra_inter_data import HeteroDCVFeatureProcessor
from NanoParticleTools.inputs.nanoparticle import SphericalConstraint
from NanoParticleTools.machine_learning.util.wandb import model_from_file

First, define the feature processor. In this case, since we are using the HeteroDCVModel, we use the HeteroDCVFeatureProcessor. We also define the label processor, which sums the wavelengths over the UCNP emisison spectra. 

In [7]:
feature_processor_kwargs={'include_zeros': True}
feature_processor = HeteroDCVFeatureProcessor(feature_processor_kwargs)

label_processor_kwargs={
        'spectrum_ranges': {
            'uv': (300, 450)
        },
        'log_constant': 100
    }
label_processor = SummedWavelengthRangeLabelProcessor(label_processor_kwargs)

Next, define an arbitrary nanoparticle. In this case it will be a core-shell nanoparticle with a 4 nm core and a 10 nm shell. The core will have Erbium doped at 0.01, while the shell will have Erbium and Ytterbium doping at 0.2 and 0.4. 

In [5]:
constraints = [SphericalConstraint(40), SphericalConstraint(100)]
dopant_concentrations = [{'Er': 0.01}, {'Er': 0.2, 'Yb': 0.4}]

From here, we can transform this definition of a UCNP into a readable data object for the model

In [16]:
dopant_concentration, radii_without_zero = feature_processor.inputs_from_constraints_and_concentration(constraints, dopant_concentrations)
input_data = feature_processor.graph_from_inputs(dopant_concentration, radii_without_zero)
data = feature_processor.data_cls(input_data)

Now, we can load a pre-trained model checkpoint, which can be downloaded from [Figshare](https://figshare.com/articles/dataset/Hetero-GNN_Checkpoints/27941694/1?file=50919813). 

In [15]:
model_path = r"C:\Users\ChemeGrad2021\Desktop\NanoParticleTools\SUNSET-1\model_1.ckpt"
pre_trained_model = model_from_file(model_path, HeteroDCVModel)

Lightning automatically upgraded your loaded checkpoint from v1.7.7 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\ChemeGrad2021\Desktop\NanoParticleTools\SUNSET-1\model_1.ckpt`
c:\users\chemegrad2021\desktop\nanoparticletools\src\NanoParticleTools\machine_learning\models\hetero\intra_inter_model.py:260: UserWarning: Cannot override n_input_nodes for this model. It is inferred fromthe embed_dim.
  warnings.warn(


Finally, we can pass the data into the model. To get emission intensity, we must undo the log and add back a constant, which were operations used in the label processor.

In [31]:
output = pre_trained_model.predict_step(data).item()
intensity = 10**(output) + label_processor.log_constant
print(f"The summed UV emission intensity of this arbitrary UCNP is: {intensity:.2f} cps")

The summed UV emission intensity of this arbitrary UCNP is: 122.25 cps
